In [24]:
import os
import cv2
import numpy as np

In [25]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

Replicate results

In [26]:
np.random.seed(42)

# ROOT_DIR = '../data/images'
TRAIN_DIR = '../train-data'
TEST_DIR = '../test-data'
input_shape = (32, 32, 1)
batch_size = 32
epochs = 30

# Build and compile model

In [27]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(3, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        640       
                                                                 
 activation_6 (Activation)   (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 32)        18464     
                                                                 
 activation_7 (Activation)   (None, 16, 16, 32)        0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                  

# Load data

In [28]:
def load_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Fit to input size
    img = cv2.resize(img, (32, 32))
    img = np.expand_dims(img, axis=-1)
    return img.astype(np.float32)

In [29]:
def one_hot_encode(labels):
    mapper = {'blank': 0, 'nough': 1, 'circle': 2}
    encoded = [mapper[label] for label in labels]
    target = to_categorical(encoded)
    return target

In [30]:
def load_data(root_dir, shuffle=True):
    X, y = [], []
    for root, dirs, files in os.walk(root_dir):
        for class_dir in dirs:
            image_dir = os.path.join(root_dir, class_dir)
            y.extend(np.tile(class_dir, len(os.listdir(image_dir))))
            for img_fn in os.listdir(image_dir):
                img = load_img(os.path.join(image_dir, img_fn))
                X.append(img)
    y = one_hot_encode(y)
    if shuffle:
        # Combine to maintain order
        data = list(zip(X, y))
        np.random.shuffle(data)
        X, y = zip(*data)
    return np.asarray(X), np.asarray(y)


In [31]:
print('Loading data...')
X_train, y_train = load_data(TRAIN_DIR)
X_test, y_test = load_data(TEST_DIR)
print('{} instances for training'.format(len(X_train)))
print('{} instances for evaluation'.format(len(X_test)))

Loading data...
480 instances for training
120 instances for evaluation


# Create more instances since our dataset is VERY limited

In [32]:
train_val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1 / 255,
    validation_split=0.2)

train_generator = train_val_datagen.flow(
    X_train, y_train, batch_size=batch_size, subset='training')

val_generator = train_val_datagen.flow(
    X_train, y_train, batch_size=batch_size, subset='validation')

# Train and evaluate

In [33]:
callbacks = [
    EarlyStopping(patience=5, verbose=1, restore_best_weights=True)]

print('Training model...')
history = model.fit_generator(
    train_generator,
    steps_per_epoch=128,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=32,
    callbacks=callbacks)

print('Evaluating model...')

Training model...
Epoch 1/30


/tmp/ipykernel_5863/1771159742.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
2023-05-01 09:36:44.861645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


128/128 [==============================] - 1s 3ms/step - loss: 0.9274 - accuracy: 0.5182 - val_loss: 0.6850 - val_accuracy: 0.6979
Evaluating model...


2023-05-01 09:36:45.438003: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


# Load data

In [34]:
test_datagen = ImageDataGenerator(rescale=1 / 255)
X_test, y_test = next(test_datagen.flow(X_test, y_test, batch_size=len(X_test)))

loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Crossentropy loss: {:0.3f}'.format(loss))
print('Accuracy: {:0.3f}'.format(acc))

4/4 [==============================] - 0s 4ms/step - loss: 0.7125 - accuracy: 0.6500
Crossentropy loss: 0.713
Accuracy: 0.650


# Save model

In [35]:
model.save('../model.h5')
print('Saved model to disk')


Saved model to disk
